# Lake Superior Data ETL

This is an ETL for gathering multiple climate variables for Lake Superior. The intention for gathering this data is to create a model to determine how water temperatures have changed over time & how various climate interactions affect water temperatures. 

Area of Interest (AOI) for this study is Lake Superior with bounding
box coordinates: lon: 92.5W to 84W lat: 46N to 49.2N

### Data from Great Lakes Observatory System via NOAA's ERDDAP (Easier access to scientific data).
http://data.glos.us/erddap/tabledap/45023.html

ERDDAP offers Tabular (tabledap) & Gridded (griddap) Datasets in a
RESTful API in addition to a files system available through curl or a browser. It has many standard file type options which can be easily
converted in the fType variable. It also has a good guide for these services: http://data.glos.us/erddap/index.html

#### This collects csv files for 13 buoys with wind speed & direction, air temp & pressure, dew point, lat & long data.

Buoys operated by Great Lakes Observing System (glos). This is not an extensive temporal dataset but has many variables. https://glbuoys.glos.us/

In [34]:
import requests as r

In [ ]:
# Make api call & verify response
csv_url = ('http://data.glos.us/erddap/tabledap')
response = r.get('csv_url')
print(response.status_code)

In [37]:
# Get a list of all available datasets for AOI in json format

response = r.get("http://data.glos.us/erddap/search/advanced.json?page=1&itemsPerPage=1000&searchFor=&protocol=%28ANY%29&cdm_data_type=%28ANY%29&institution=%28ANY%29&ioos_category=%28ANY%29&keywords=%28ANY%29&long_name=%28ANY%29&standard_name=%28ANY%29&variableName=%28ANY%29&maxLat=49&minLon=-84&maxLon=-92.5&minLat=46&minTime=&maxTime=")
print(response.json()) # Return results in a json


{'table': {'columnNames': ['griddap', 'Subset', 'tabledap', 'Make A Graph', 'files', 'Title', 'Summary', 'FGDC', 'ISO 19115', 'Info', 'Background Info', 'RSS', 'Institution', 'Dataset ID'], 'columnTypes': ['String', 'String', 'String', 'String', 'String', 'String', 'String', 'String', 'String', 'String', 'String', 'String', 'String', 'String'], 'rows': [['', '', 'http://data.glos.us/erddap/tabledap/45023', 'http://data.glos.us/erddap/tabledap/45023.graph', '', '45023', '45023. Data from a local source.\n\ncdm_data_type = TimeSeries\nVARIABLES:\ntime (time of observation, seconds since 1970-01-01T00:00:00Z)\nstation_name\nlatitude (station latitude, degrees_north)\nlongitude (station longitude, degrees_east)\ndepth (depth of the sensor relative to sea surface, m)\nThermistor_String_at_32m (Thermistor Temperature, celsius)\nThermistor_String_at_13m (Thermistor Temperature, celsius)\nsea_surface_wave_mean_height_of_highest_tenth (m)\nThermistor_String_at_11m (Thermistor Temperature, celsi

In [38]:
# Build a url to get all variables & time. If nothing is specified it returns everything.
base = 'http://data.glos.us/erddap/tabledap/'
buoy = ['45027', '45028', '45171', '45172', '45179', 'GLRCMET', 'GRIM4', 'MTRI-NKW', 'MTRI-SKW', 'STDM4', '45023', '45023_profile','45025', '45173'] 
fType = '.csv'

In [45]:
# Iterate thru the list of Lake Superior buoys & copy files to disc

for i in buoy:
    url = (base + i + fType)
    req = r.get(url)
    url_content = req.content
   
    for iname in buoy:
        fName = (iname + fType)
        # Write the file to disc
        with open((r'C:\Users\mmMary\Documents\Projects\Superior_Surface_Temp\Data\GLOS_Buoy/'+ fName), 'wb') as zip:
            zip.write(req.content)
       

 ## <font color='purple'>---------------------------------------------------------------

### This section collects Glider data operated by Univeristy of MN Duluth & Large Lakes Observatory (LLO). Data accessed via ERDDAP with erddapy package & saved as csv files.

https://gliders.ioos.us/erddap/tabledap/index.html?page=1&itemsPerPage=1000 <br> https://www.ncei.noaa.gov/access/integrated-ocean-observing-system/ <br> https://github.com/ioos/erddapy

In [3]:
# Install erddapy in order to access ERDDAP API
!pip install erddapy

In [1]:
# Module needed to work with dataframe
import pandas

In [4]:
# Import erddapy package
from erddapy import ERDDAP

# Call the server
req = ERDDAP(
    server="NGDAC", # "NOAA IOOS NGDAC (National Glider Data Assembly Center)"
    protocol="tabledap",
    response="nc",
)

In [ ]:
# Define the dataset you want. Change the name here.
dataset =  	'gp_363-20160630T0230'

# Set the search parameters. This is very specific to the dataset being pulled.

# Additional search criteria can be specified following the below example 
# constraints. Otherwise leave them commented out to get entire dataset.
req.response = "nc"
req.dataset_id = dataset
# e.constraints = {
#     "time>=": "2011-01-01T00:00:00Z",
#     "time<=": "2011-12-01T00:00:00Z",
#     "latitude>=": 46.0,
#     "latitude<=": 49.2,
#     "longitude>=": -84.0,
#     "longitude<=": -92.5,
# }

# Set the variables of interest. There are many others available.
req.variables = [
    "depth",
    "precise_lat",
    "precise_lon",
    "temperature",
    "precise_time",
]

# Put the data into a pandas dataframe
df = req.to_pandas()

# Delete rows with null values
df = df.dropna(how='any',axis=0) 

# Save cleaned data as csv
df.to_csv(dataset + '.csv', index = True)

In [10]:
# View file contents
df

,depth (m),precise_lat (degree_north),precise_lon (degree_east),temperature (Celsius),precise_time (UTC)
0,0.198408,43.305421,-79.074776,10.2090,2018-05-23T14:42:05Z
1,NaN,43.305422,-79.074775,NaN,2018-05-23T14:42:05Z
2,NaN,43.305426,-79.074771,NaN,2018-05-23T14:42:06Z
3,0.198408,43.305430,-79.074767,10.2043,2018-05-23T14:42:07Z
4,NaN,43.305435,-79.074763,NaN,2018-05-23T14:42:08Z
...,...,...,...,...,...
1850342,0.119044,43.345671,-78.818134,8.8063,2018-06-12T22:21:13Z
1850343,NaN,43.345673,-78.818129,NaN,2018-06-12T22:21:15Z
1850344,0.109124,43.345674,-78.818127,8.8179,2018-06-12T22:21:15Z
1850345,0.089283,43.345676,-78.818120,8.8252,2018-06-12T22:21:17Z


In [7]:
# Delete NaN rows
df = df.dropna(how='any',axis=0) 
df

,depth (m),precise_lat (degrees_north),precise_lon (degrees_east),temperature (degree_Celsius),precise_time (UTC)
0,0.069399,50.167376,-144.499390,10.7315,2016-06-30T02:37:32Z
1,1.774620,50.167463,-144.499366,10.7240,2016-06-30T02:38:33Z
2,12.035389,50.167450,-144.499302,10.6842,2016-06-30T02:39:33Z
3,0.118969,50.167753,-144.496872,10.7472,2016-06-30T03:05:48Z
4,5.006570,50.167838,-144.497292,10.7315,2016-06-30T03:06:49Z
...,...,...,...,...,...
3205,25.378666,50.128460,-144.411606,10.7335,2016-07-04T21:20:40Z
3206,31.514675,50.128460,-144.411606,10.3793,2016-07-04T21:21:40Z
3207,36.827774,50.128460,-144.411606,9.7921,2016-07-04T21:22:40Z
3208,41.674870,50.128460,-144.411606,9.2827,2016-07-04T21:23:40Z


In [ ]:
# Export cleaned dataframe as csv
df.to_csv(dataset + '.csv', index = True)

Delete null values from the df:
- https://stackoverflow.com/questions/44548721/remove-row-with-null-value-from-pandas-data-frame
- https://www.geeksforgeeks.org/how-to-export-pandas-dataframe-to-a-csv-file/
- https://www.youtube.com/watch?v=Y_f0L62TleE -->

 ## <font color='purple'>---------------------------------------------------------------

### Access data from NOAA FTP
https://psl.noaa.gov/data/gridded/data.cobe.html <br>
https://www.ncei.noaa.gov/access/integrated-ocean-observing-system/ <br>
Index of files ftp://ftp.cdc.noaa.gov/Datasets/

This collects satellite derived Sea Surface Monthly Mean Temp data from NOAA from January 2006 to December 2009 in a netCDF file. <br>

In [ ]:
# !pip install xarray

In [2]:
# Modules for working with FTP & netCDF files
from ftplib import FTP 

# import netCDF4
import xarray as xr

In [12]:
# Connect to ftp server

ftp = FTP('ftp2.psl.noaa.gov')  # Connect to host with default port
ftp.login()                     # Use anonymous credentials
'230 Login successful.'


'230 Login successful.'

In [13]:
# Check directory contents
ftp.cwd('/Datasets/COBE/')  # Change into "COBE" directory
ftp.retrlines('LIST')           # List directory contents

-rw-rw-r--    1 2207     20002        3640 Jul 12  2016 README
-rw-rw-r--    1 2207     20002     4670580 Feb 09 08:56 sst.mon.1991-2020.ltm.nc
-rw-rw-r--    1 2207     20002     1582748 Apr 13  2020 sst.mon.ltm.1981-2010.nc
-rw-rw-r--    1 2207     20002    161738746 Apr 07 09:14 sst.mon.mean.nc


'226 Directory send OK.'

In [15]:
# Download as binary 
with open('sst.mon.mean.nc', 'wb') as fp:    
    ftp.retrbinary('RETR sst.mon.mean.nc', fp.write)  # FTP's RETR command downloads file

#ftp.quit()  # Can quit when transfers are complete

In [3]:
# Take a peek at the file
sst = xr.open_dataset(r'C:\Users\mmMary\Documents\Projects\Superior_Surface_Temp\sst.mon.mean.nc')
sst

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 1563)
Coordinates:
  * lat      (lat) float32 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * lon      (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * time     (time) datetime64[ns] 1891-01-01 1891-02-01 ... 2021-03-01
Data variables:
    sst      (time, lat, lon) float32 ...
Attributes:
    title:            COBE Sea Surface Temperature Analysis
    history:          created June 2011 at ESRL/PSD using the grib data from ...
    platform:         Analyses
    original_source:  http://near-goos1.jodc.go.jp/cgi-bin/1997/near_goos_cat...
    Conventions:      CF-1.2
    institution:      NOAA ESRL/PSD
    comment:          recent values (w/i last year) may change as dataset is ...
    dataset_title:    COBE Sea Surface Temperature
    References:       https://www.psl.noaa.gov/data/gridded/data.cobe.html